In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
!pip install emot --upgrade
!pip install emoji
!pip install demoji
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.8/361.8 kB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for emoji: filename=emoji-2.7.0-py2.py3-none-any.whl size=356563 sha256=b6968d2f2919cc4607f44e76b67732430f3833e5d94fb91ebf44b5122751d96b
  Stored in directory: /root/.cache/pip/wheels/41/11/48/5df0b9727d5669c9174a141134f10304d1d78a3b89a4676f3d
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 15.5 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
import emot
import nltk.data
import numpy as np
import pandas as pd
import re
import time
import string
import emoji
import demoji
import contractions
import nltk
nltk.download('omw-1.4')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option("max_colwidth" ,220)
wn = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

<ipython-input-6-e00aa54160fe>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
demoji.download_codes()
def emo(text):
  try:
   temp=emoji.demojize(str(text),delimiters=(" "," "))
   temp=temp.replace("_","  ")
   return temp
  except IndexError:
        return ""

<ipython-input-7-82a5fb4117d1>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
punct="!#$%&\'()*+,-.’’/:;<=>?@[\\]^_`{|}~’“‘"

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', str(text))

In [ ]:
def clean_text(text):
    text = re.sub('@[a-zA-Z0-9]*', '', str(text))#to remove @ and its following word
    text = contractions.fix(text, slang=True)
    text = "".join([word.lower() for word in text if word not in string.punctuation]) #to remove punctuation
    text="".join([word.lower() for word in text if word not in punct])
    text = "".join([word for word in text if not word.isdigit()])#to remove digit
    text = " ".join(word for word in text.split() if word not in stopwords)
    return text

In [ ]:
bodo_train=pd.read_csv('/content/drive/MyDrive/hasoc/task4/bodo/train_BO_AH_HASOC2023.csv')
bodo_test=pd.read_csv('/content/drive/MyDrive/hasoc/task4/bodo/test_BO_AH_HASOC2023.csv')

In [ ]:
len(bodo_train)

1679

In [ ]:
bodo_train['task_1'].value_counts()

HOF    998
NOT    681
Name: task_1, dtype: int64

In [ ]:
bodo_train['clean_text'] = bodo_train['text'].apply(lambda x:emo(x))
bodo_train['clean_text'] = bodo_train['clean_text'].apply(lambda x:remove_urls(x))
bodo_train['clean_text'] = bodo_train['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
bodo_train['clean_text']

0                                                                                                 गोदाव खामानि मावओ बोला नो सानसे देरहा थारगोन
1                                                                                                                      निखावरि सुबुंफोरा सिखाव
2                                                                                                   मा बिमा खर परिबर्थननि खोथा फैखो बेयाव मोसौ
3                                                                                                      थोद जामबा सैमा साला मा मिसेस जाखो बेलाय
4                                                      माखौ बकिबाय थादों नों बोरमा फानथा दम दंब्ला खामानि मावना दिन्थि बिनि उनाव बखि सैमा साला
                                                                         ...                                                                  
1674                                                                   नोंलाय जामबा नोंबो सासे सनमान गैयै मानसिसो गिदिरसिन मानसिखौ मुं मुखग्रा

In [ ]:
bodo_test['clean_text'] = bodo_test['text'].apply(lambda x:emo(x))
bodo_test['clean_text'] = bodo_test['clean_text'].apply(lambda x:remove_urls(x))
bodo_test['clean_text'] = bodo_test['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
bodo_train['clean_text']

0                                                                                                 गोदाव खामानि मावओ बोला नो सानसे देरहा थारगोन
1                                                                                                                      निखावरि सुबुंफोरा सिखाव
2                                                                                                   मा बिमा खर परिबर्थननि खोथा फैखो बेयाव मोसौ
3                                                                                                      थोद जामबा सैमा साला मा मिसेस जाखो बेलाय
4                                                      माखौ बकिबाय थादों नों बोरमा फानथा दम दंब्ला खामानि मावना दिन्थि बिनि उनाव बखि सैमा साला
                                                                         ...                                                                  
1674                                                                   नोंलाय जामबा नोंबो सासे सनमान गैयै मानसिसो गिदिरसिन मानसिखौ मुं मुखग्रा

In [ ]:
bodo_train.shape

(1679, 4)

In [ ]:
bodo_test.shape

(420, 3)

In [ ]:
bodo_test.head()

,S. No.,text,clean_text
0,1,BPF बानाय लांनाय लामाया 5 बोसोरानो जोरासै,bpf बानाय लांनाय लामाया बोसोरानो जोरासै
1,2,"बै समाव माबेयाव हाबसोनानै दंमोन नोंलाय, दाना बुजिना जाथारबाय",बै समाव माबेयाव हाबसोनानै दंमोन नोंलाय दाना बुजिना जाथारबाय
2,3,"बे थांखिखौ मिनिग्रापोरा हारिखौ लेवारपोरबायदि थै सोबग्रासो। राजनिथिक गादिनि गोदै गोबाब स्वाद मोनजानानै गावनि हारिखौ निन्दिग्रा, मिनिग्रा।",बे थांखिखौ मिनिग्रापोरा हारिखौ लेवारपोरबायदि थै सोबग्रासो। राजनिथिक गादिनि गोदै गोबाब स्वाद मोनजानानै गावनि हारिखौ निन्दिग्रा मिनिग्रा।
3,4,मोसौ खुगायाव एमफौ नांबाय नोंनाव सैमा,मोसौ खुगायाव एमफौ नांबाय नोंनाव सैमा
4,5,2003आव BTC गोरोबथा जादों बेनि थाखाय बो थोजासे स'म होदों नाथाय राज्यो-राजाफोरखौ,आव btc गोरोबथा जादों बेनि थाखाय बो थोजासे सम होदों नाथाय राज्योराजाफोरखौ


In [ ]:
X_train=bodo_train['clean_text']
X_test=bodo_test['clean_text']
y_train=bodo_train['task_1']

In [ ]:
X_train.shape, X_test.shape

((1679,), (420,))

In [ ]:
from sklearn.feature_extraction.text import  CountVectorizer, TfidfTransformer,TfidfVectorizer

In [ ]:
!git clone https://github.com/libindic/syllabalizer.git
%cd syllabalizer
%run setup.py sdist

Cloning into 'syllabalizer'...
remote: Enumerating objects: 186, done.
remote: Total 186 (delta 0), reused 0 (delta 0), pack-reused 186
Receiving objects: 100% (186/186), 41.35 KiB | 10.34 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/syllabalizer


/usr/local/lib/python3.10/dist-packages/setuptools/dist.py:755: SetuptoolsDeprecationWarning: Invalid dash-separated options
!!

        ********************************************************************************
        Usage of dash-separated 'description-file' will not be supported in future
        versions. Please use the underscore name 'description_file' instead.

        By 2023-Sep-26, you need to update your project and remove deprecated calls
        or your builds will no longer be supported.

        See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
        ********************************************************************************

!!
  opt = self.warn_dash_deprecation(opt, section)
/usr/local/lib/python3.10/dist-packages/setuptools/dist.py:755: SetuptoolsDeprecationWarning: Invalid dash-separated options
!!

        ********************************************************************************
        Usage of dash-separa

In [ ]:
!pip install dist/libindic-syllabifier*.tar.gz
!pip install libindic-utils

Processing ./dist/libindic-syllabifier-0.0.1.dev28.tar.gz
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for libindic-syllabifier: filename=libindic_syllabifier-0.0.1.dev28-py2.py3-none-any.whl size=8121 sha256=845c0823e05bed626a07632a85855539c95da5b462cadbee52ef4807d19b55c2
  Stored in directory: /root/.cache/pip/wheels/d2/50/19/e2620530fc718493df9f5bca90d23db025cc864b9c7c0a98ef
  Created wheel for silpa_common: filename=silpa_common-0.3-py3-none-any.whl size=8466 sha256=9f78655c0b76ae0ac4dbc9cbc9a639dc003c2d77b60eae2acc854612012c5442
  Stored in directory: /root/.cache/pip/wheels/c0/72/43/0c779b79d708c78240beb3b0bb8f5ff3c2ab81c4e5271ea1aa
Successfully built libindic-syllabifier silpa_common
  Preparing metadata (setup.py) ... done
  Created wheel for libindic-utils: filename=libindic_utils-1.0.3-py3-none-any.whl size=9686 sha256=a4f73fa5c571883114145673d952d319fa3fbda7718d6a18e55a106d7f477d4c
  Stored in directory: /root/.cache/pip/w

In [ ]:
from libindic.syllabifier import Syllabifier
instance = Syllabifier()

In [ ]:
def syllable(word):
    try:
        sylbls = instance.syllabify(word)
    except:
        sylbls = list(word)
    return sylbls

In [ ]:
# generate word n-grams

def generate_N_grams(text, ngram=1):
    words = [word for word in text.split(" ")]  # if word not in set(stopwords.words('english'))]
    temp = zip(*[words[i:] for i in range(0, ngram)])
    ngrams = [' '.join(ngram) for ngram in temp]
    return ngrams

In [ ]:
import regex

def custom_analyzer(text):
    words = regex.findall(r'\w{2,}', text) # extract words of at least 2 letters
    for w in words:
        yield w

In [ ]:
# generate syllable n-grams by specifying the value of n, by default ngram=1

def generate_cN_grams(word, ngram=1):
    sylbs = [sylbs for sylbs in syllable(word)]  # if word not in set(stopwords.words('english'))]
    temp = zip(*[sylbs[i:] for i in range(0, ngram)])
    ngrams = [''.join(ngram) for ngram in temp]
    return ngrams

In [ ]:
# generate syllable n-grams in the range 'from' to 'to'. by default 'from'= 1

def ngrams(text, r_to, r_from=1):
    r_from = 1
    r_to = 3
    allngrams=[]
    for word in text.split():
        temp = []
        for i in range(r_from + r_to - 1):
            temp.extend(generate_cN_grams(word, i+1))
            allngrams.append(temp)
    n_grams = [item for sublist in allngrams for item in sublist]
  # merged = list(itertools.chain(allngrams))
    return n_grams

In [ ]:
def custom_analyzer(text):
    syngrams = ngrams(text,3)
    return syngrams

In [ ]:
tfidf_vectorizer = TfidfVectorizer(analyzer=custom_analyzer)
tfidf_vectorizer.fit_transform(X_train)
xtrain_tfidf =  tfidf_vectorizer.transform(X_train)
xtest_tfidf =  tfidf_vectorizer.transform(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC

clf1 = LogisticRegression(random_state=1)
clf2 = BernoulliNB()
clf3 = LinearSVC(penalty='l2',C=1.0)

eclf = VotingClassifier(estimators=[('LR', clf1), ('BNB', clf2),
('SVC', clf3)], voting='hard')
eclf = eclf.fit(xtrain_tfidf, y_train)
y_pred1= eclf.predict(xtest_tfidf)
print(y_pred1)

['NOT' 'NOT' 'HOF' 'HOF' 'NOT' 'NOT' 'HOF' 'HOF' 'HOF' 'HOF' 'NOT' 'NOT'
 'HOF' 'NOT' 'NOT' 'HOF' 'HOF' 'HOF' 'HOF' 'NOT' 'HOF' 'HOF' 'NOT' 'HOF'
 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'NOT' 'HOF' 'HOF' 'HOF' 'HOF'
 'HOF' 'NOT' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'NOT' 'HOF' 'HOF' 'HOF' 'NOT'
 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'NOT' 'HOF' 'NOT' 'HOF' 'NOT' 'HOF' 'HOF'
 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'NOT' 'HOF' 'HOF' 'HOF' 'HOF'
 'HOF' 'HOF' 'HOF' 'HOF' 'NOT' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF'
 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'NOT'
 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF'
 'HOF' 'HOF' 'HOF' 'NOT' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'NOT'
 'HOF' 'HOF' 'HOF' 'NOT' 'HOF' 'NOT' 'NOT' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF'
 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'NOT' 'HOF'
 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'NOT' 'HOF' 'HOF' 'NOT'
 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'NOT' 'HOF' 'HOF' 'H

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df= pd.DataFrame(data=y_pred1, columns=['y_pred1'])
Submisssion_assamese = pd.DataFrame()
Submisssion_assamese['S. No.'] = bodo_test['S. No.']
Submisssion_assamese['task_1'] = y_pred_df
Submisssion_assamese.to_csv('/content/drive/MyDrive/MUCS_test_bodo_run_3.csv',index = None)

In [ ]:
from sklearn.svm import SVC
svm = SVC()

svm .fit(xtrain_tfidf, y_train)

y_pred2 = svm.predict(xtest_tfidf)

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df= pd.DataFrame(data=y_pred2, columns=['y_pred2'])
Submisssion_assamese = pd.DataFrame()
Submisssion_assamese['S. No.'] = bodo_test['S. No.']
Submisssion_assamese['task_1'] = y_pred_df
Submisssion_assamese.to_csv('/content/drive/MyDrive/MUCS_test_bodo_run_4.csv',index = None)

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

logreg .fit(xtrain_tfidf, y_train)

y_pred3 = logreg .predict(xtest_tfidf)

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df= pd.DataFrame(data=y_pred3, columns=['y_pred3'])
Submisssion_assamese = pd.DataFrame()
Submisssion_assamese['S. No.'] = bodo_test['S. No.']
Submisssion_assamese['task_1'] = y_pred_df
Submisssion_assamese.to_csv('/content/drive/MyDrive/MUCS_test_bodo_run_5.csv',index = None)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier()
RFC .fit(xtrain_tfidf, y_train)

y_pred4 = RFC .predict(xtest_tfidf)

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df= pd.DataFrame(data=y_pred3, columns=['y_pred3'])
Submisssion_assamese = pd.DataFrame()
Submisssion_assamese['S. No.'] = bodo_test['S. No.']
Submisssion_assamese['task_1'] = y_pred_df
Submisssion_assamese.to_csv('/content/drive/MyDrive/MUCS_test_bodo_run_6.csv',index = None)